### Import necessary libraries

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns

from platform import python_version

import numpy as np
import scipy.io
import pandas as pd
import sklearn
import matplotlib

import timeit
import random
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from sklearn.metrics import r2_score
from sklearn import linear_model
from scipy import stats

import csv

In [2]:
print("python version==%s" % python_version())
print("pandas==%s" % pd.__version__)
print("numpy==%s" % np.__version__)
print("sklearn==%s" % sklearn.__version__)
print("torch==%s" % torch.__version__)
print("matplotlib==%s" % matplotlib.__version__)

plt.rcParams[
    "figure.facecolor"
] = "w"  # force white background on plots when using dark mode in JupyterLab


if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

python version==3.7.9
pandas==1.2.2
numpy==1.19.2
sklearn==0.24.1
torch==1.9.0
matplotlib==3.3.4


### Processing and loading data

In [3]:
comms_training=[ 'PC-BV-BT-BC-BP-EL-FP-CH-AC-BH-CG-ER-RI-DF',
                 'BV-BF-BO-BT-BU-DP-BL-BA-BP-EL-FP-CH-AC-BH-CG-ER-RI',
                 'PJ-BO-BT-BC-BY-BA-CA-EL-FP-CH-AC-BH-CG-CC-DF',
                 'PC-PJ-BF-BO-BU-DP-BA-EL-CH-BH-CG',
                 'PC-PJ-BV-BF-BT-BL-CA-EL-FP-CH-BH-CC-DF',
                 'PJ-BC-BY-BA-CA-EL-FP-AC-ER-CC-DF',
                 'PJ-BV-BO-BT-BY-BU-CA-EL-AC-CG-DF',
                 'PC-PJ-BT-BY-BU-DP-BA-CA-FP-CH-BH-CG-ER-RI-CC-DL-DF',
                 'BV-BU-DP-BA-CA-EL-FP-CH-CG-RI-DL-DF',
                 'PC-PJ-BF-BC-BU-DP-CA-EL-FP-CH-AC-CG',
                 'BO-BC-BY-BU-BA-BP-CH-AC-CG-CC-DL',
                 'PC-BV-BF-BT-BL-CA-CH-BH-CG-ER-RI-CC-DL',
                 'BC-BU-DP-BL-BP-CA-EL-FP-BH-DL-DF',
                 'PJ-BF-BC-BU-BP-CA-EL-FP-CH-AC-CG',
                 'BO-BC-BY-BU-BA-CA-EL-AC-RI-DL-DF',
                 'BV-BF-BY-BU-DP-BL-CA-EL-CH-BH-ER',
                 'BV-BF-BC-BY-BU-DP-BL-EL-CH-AC-CG',
                 'PJ-BF-DP-BL-BA-BP-CA-EL-BH-ER-DF',
                 'PJ-BV-BF-BT-BC-BY-BU-CH-AC-RI-DF',
                 'PC-PJ-BC-BY-BU-CA-EL-AC-BH-ER-RI',
                 'BY-BU-BP-CA-EL-FP-AC-BH-CG-ER-RI',
                 'PJ-BV-BF-BO-BU-DP-BL-CA-EL-FP-CH-AC',
                 'PC-BF-BO-BT-BC-BU-BP-CA-EL-AC-BH-RI',
                 'BF-BT-BC-BY-BL-BA-BP-EL-FP-AC-BH-CG-ER',
                 'PC-BF-BT-BC-BU-DP-BL-BA-FP-AC-CC-DL-DF',
                 'BC-BU-DP-BA-BP-CA-FP-CG-RI-CC-DF',
                 'PC-PJ-BV-BO-BC-BY-DP-BL-BA-BP-CA-EL-AC-BH-CG-ER-RI',
                 'PC-PJ-BV-BO-BY-BU-DP-BL-CA-EL-FP-CH-BH-CG-CC-DL',
                 'BV-BF-BL-CA-FP-CH-AC-BH-ER-RI-DF',
                 'PC-PJ-BO-BT-BC-BL-EL-FP-BH-CG-RI-DL',
                 'PJ-BV-BO-BC-BU-DP-BL-EL-CH-CG-RI',
                 'PC-PJ-BF-BT-BC-BY-DP-BL-BA-FP-AC-BH-RI-CC-DL-DF',
                 'PC-PJ-BV-BF-BT-BC-DP-CA-EL-CH-AC-BH-CG-RI-DF',
                 'BF-BU-DP-BL-BA-CA-EL-FP-AC-CG-RI',
                 'PC-BV-BF-BO-BC-DP-BL-EL-CH-BH-CG-RI',
                 'PC-BV-BF-BO-CA-EL-FP-CH-AC-BH-ER',
                 'PJ-BV-BY-BA-EL-FP-CH-BH-ER-DL-DF',
                 'PJ-BV-BF-BU-DP-BL-BA-EL-RI-DL-DF',
                 'BV-BO-BC-BY-BL-BA-FP-AC-BH-CG-DL-DF',
                 'PJ-BY-BU-EL-FP-CH-AC-BH-CG-CC-DF',
                 'PC-PJ-BO-BT-BL-BP-CH-AC-BH-CG-CC-DF',
                 'PJ-BO-BT-BU-BP-CA-EL-CH-AC-BH-RI',
                 'PC-PJ-BF-BU-DP-BA-CA-FP-CH-AC-RI-CC-DL',
                 'PJ-BV-BO-BT-BY-BU-DP-BL-CA-BH-CG-ER-CC-DL',
                 'BV-BF-BO-BY-BU-DP-CA-EL-CH-RI-DF',
                 'BF-BO-BT-BC-BU-CA-EL-AC-ER-RI-DF',
                 'PJ-BF-BC-BU-BA-BP-CA-EL-AC-BH-DL',
                 'PC-PJ-BV-BF-DP-EL-FP-CH-AC-ER-CC-DF',
                 'PC-PJ-BT-BC-BU-DP-BL-BP-CA-EL-RI-CC-DF',
                 'PC-BF-BT-BC-BU-DP-BL-CH-BH-RI-CC',
                 'PC-PJ-BV-BT-BC-BA-CA-CH-AC-ER-RI-CC-DL',
                 'BF-BO-BC-BL-CA-EL-BH-CG-RI-CC-DL',
                 'BY-BU-DP-BL-CA-EL-CH-BH-CG-CC-DF',
                 'PJ-BV-BF-BT-BC-BY-DP-CH-AC-BH-CC-DF',
                 'PC-PJ-BF-BT-BC-BU-DP-BL-CH-AC-BH-CG-RI-DF',
                 'PJ-BV-BO-BT-BC-BY-BP-CA-EL-FP-AC-BH-CG',
                 'PJ-BC-BP-EL-FP-AC-BH-ER-RI-CC-DF',
                 'BV-BO-BC-BU-DP-BP-EL-FP-AC-RI-DL-DF',
                 'PC-BF-BO-BU-BA-BP-CA-FP-CG-CC-DL-DF',
                 'BC-BU-BL-CA-EL-FP-AC-ER-RI-CC-DL']

comms_validation=[   'PC-PJ-BV-BF-BC-BY-BU-DP-BL-BA-BP-CA-FP-BH-ER-RI-CC-DL-DF',
                     'PC-PJ-BV-BO-BC-BY-BU-BP-EL-FP-AC-BH-CG-ER',
                     'PC-BV-BF-BY-BU-EL-FP-CH-AC-ER-DF',
                     'PC-BF-BO-BT-BC-BY-DP-BL-BP-CA-FP-CH-AC-BH-ER-RI-DL-DF',
                     'BF-BC-BY-BA-CA-EL-FP-CH-AC-BH-ER-CC-DL-DF',
                     'BV-BO-BT-BY-BL-BA-BP-EL-FP-AC-BH-DL-DF',
                     'PC-PJ-BV-BF-BY-BU-BA-EL-FP-CH-AC-CG-RI',
                     'PC-PJ-BV-BO-BT-BY-BA-BP-EL-FP-CH-CG-DL',
                     'PC-PJ-BL-BA-BP-CA-EL-FP-CH-AC-ER-RI',
                     'PC-PJ-BF-BY-DP-BA-CA-CG-RI-CC-DL',
                     'PC-BO-BC-BU-DP-BL-CA-AC-BH-CG-RI-CC-DF',
                     'PC-PJ-BF-BO-BY-BU-AC-BH-RI-DL-DF',
                     'PJ-BV-BF-BO-BT-BC-BU-BA-CA-BH-ER-CC-DF',
                     'PC-PJ-BV-BO-BC-BU-BL-BA-CA-FP-CH-BH-CG-CC',
                     'PC-PJ-BC-BU-BL-BP-CA-EL-FP-AC-CG-DL',
                     'PJ-BF-BO-BU-BA-CA-EL-CH-BH-CG-RI-CC-DL',
                     'PC-BF-BT-DP-BL-CA-CH-AC-CG-ER-DF',
                     'PC-PJ-BV-BF-BY-BA-BP-CA-EL-BH-CG-ER-DL',
                     'PC-PJ-BO-BT-BC-BY-FP-CH-AC-ER-RI-DL-DF',
                     'BF-BO-BC-BY-BU-DP-BA-BP-FP-CH-AC-BH-ER-RI-CC-DL-DF',
                     'PJ-BV-BO-BU-CA-CH-AC-BH-CG-RI-DL-DF',
                     'PC-BF-BO-BC-BY-DP-BL-BA-BP-CA-EL-AC-BH-CG-RI-CC-DL',
                     'PJ-BO-BT-BC-CA-EL-FP-AC-BH-ER-RI',
                     'PJ-BO-BT-BC-BY-BU-BL-BP-FP-AC-BH-CG-RI-DF',
                     'PC-PJ-BC-BY-DP-BL-CA-FP-CH-BH-ER-RI',
                     'BF-BU-BL-BA-EL-FP-CH-BH-CG-RI-DL',
                     'PC-PJ-BO-BC-BY-BU-DP-BL-CA-EL-CH-AC-BH-CG-DL-DF',
                     'PC-PJ-BV-BF-BO-BU-CA-CH-AC-BH-CG',
                     'PJ-BV-BT-BC-BY-BU-DP-BA-BP-CH-AC-CG-CC-DF',
                     'PC-BT-BC-BY-DP-BL-BA-CA-FP-CH-BH-CG-ER-RI',
                     'PC-PJ-BF-BO-BU-DP-BA-CA-EL-FP-CH-AC-BH-ER-CC-DL-DF',
                     'PC-BV-BF-BO-BT-BU-DP-CA-EL-FP-BH-CG-DF',
                     'PJ-BF-BO-BT-BC-BY-BU-DP-BL-BA-CH-AC-CG-RI-CC-DF',
                     'PJ-BV-BO-BU-CA-EL-FP-CH-BH-CG-ER',
                     'PC-PJ-BV-BF-BO-BT-BC-BY-BU-DP-BL-BA-BP-CA-EL-FP-CH-AC-BH-CG-ER-RI-CC-DL-DF']

In [4]:
train_abundance = {}
test_abundance  = {}

buty_train  = {}
ace_train   = {}
lac_train   = {}
succ_train  = {}

buty_test   = {}
ace_test    = {}
lac_test    = {}
succ_test   = {}

with open('2021_02_19_MultifunctionalDynamicData.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    four_count = 1
    butyrate, acetate, lactate, succinate = [], [], [], []
    abundance_data                        = []
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
        else:
            comm  = row[1]
            time  = float(row[2])
            PC_OD = float(row[6])
            PJ_OD = float(row[8])
            BV_OD = float(row[10])
            BF_OD = float(row[12])
            BO_OD = float(row[14])
            BT_OD = float(row[16])
            BC_OD = float(row[18])
            BY_OD = float(row[20])
            BU_OD = float(row[22])
            DP_OD = float(row[24])
            BL_OD = float(row[26])
            BA_OD = float(row[28])
            BP_OD = float(row[30])
            CA_OD = float(row[32])
            EL_OD = float(row[34])
            FP_OD = float(row[36])
            CH_OD = float(row[38])
            AC_OD = float(row[40])
            BH_OD = float(row[42])
            CG_OD = float(row[44])
            ER_OD = float(row[46])
            RI_OD = float(row[48])
            CC_OD = float(row[50])
            DL_OD = float(row[52])
            DF_OD = float(row[54])
            buty  = float(row[56])
            ace   = float(row[57])
            lac   = float(row[58])
            succ  = float(row[59])
            
            butyrate.append(buty)
            acetate.append(ace)
            lactate.append(lac)
            succinate.append(succ)
            
            abundance_data.append([PC_OD, PJ_OD, BV_OD, BF_OD, BO_OD, BT_OD, \
                                   BC_OD, BY_OD, BU_OD, DP_OD, BL_OD, BA_OD, \
                                   BP_OD, CA_OD, EL_OD, FP_OD, CH_OD, AC_OD, \
                                   BH_OD, CG_OD, ER_OD, RI_OD, CC_OD, DL_OD, DF_OD])
            
            if comm in comms_training:
                is_train = True
            else:
                is_train = False
                
            if is_train:
                if comm not in train_abundance.keys():
                    train_abundance[comm] = []
                    buty_train[comm]      = []
                    ace_train[comm]       = []
                    lac_train[comm]       = []
                    succ_train[comm]      = []
            else:
                if comm not in test_abundance.keys():
                    test_abundance[comm] = []
                    buty_test[comm]      = []
                    ace_test[comm]       = []
                    lac_test[comm]       = []
                    succ_test[comm]      = []
            
            if four_count == 4:
                if is_train:
                    train_abundance[comm] = np.array(abundance_data)
                    buty_train[comm]      = np.array(butyrate)
                    ace_train[comm]       = np.array(acetate)
                    lac_train[comm]       = np.array(lactate)
                    succ_train[comm]      = np.array(succinate)
                else:
                    test_abundance[comm]  = np.array(abundance_data)
                    buty_test[comm]       = np.array(butyrate)
                    ace_test[comm]        = np.array(acetate)
                    lac_test[comm]        = np.array(lactate)
                    succ_test[comm]       = np.array(succinate)
                
                four_count = 1
                abundance_data                        = []
                butyrate, acetate, lactate, succinate = [], [], [], []
            else:
                four_count += 1
        line_count += 1
    print(f'Processed {line_count} lines.')

Column names are , Community, Time, Richness, Contamination, PC, PC_OD, PJ, PJ_OD, BV, BV_OD, BF, BF_OD, BO, BO_OD, BT, BT_OD, BC, BC_OD, BY, BY_OD, BU, BU_OD, DP, DP_OD, BL, BL_OD, BA, BA_OD, BP, BP_OD, CA, CA_OD, EL, EL_OD, FP, FP_OD, CH, CH_OD, AC, AC_OD, BH, BH_OD, CG, CG_OD, ER, ER_OD, RI, RI_OD, CC, CC_OD, DL, DL_OD, DF, DF_OD, OD, Butyrate, Acetate, Lactate, Succinate
Processed 381 lines.


In [5]:
train_abundance_data = np.zeros((len(train_abundance),4,25))
test_abundance_data  = np.zeros((len(test_abundance),4,25))

train_butyrate  = np.zeros((len(train_abundance),4,1))
train_acetate   = np.zeros((len(train_abundance),4,1))
train_lactate   = np.zeros((len(train_abundance),4,1))
train_succinate = np.zeros((len(train_abundance),4,1))

test_butyrate   = np.zeros((len(test_abundance),4,1))
test_acetate    = np.zeros((len(test_abundance),4,1))
test_lactate    = np.zeros((len(test_abundance),4,1))
test_succinate  = np.zeros((len(test_abundance),4,1))

for no, comm in enumerate(train_abundance):
    train_abundance_data[no] = train_abundance[comm].reshape(1,4,25)
    train_butyrate[no]       = buty_train[comm].reshape(1,4,1)
    train_acetate[no]        = ace_train[comm].reshape(1,4,1)
    train_lactate[no]        = lac_train[comm].reshape(1,4,1)
    train_succinate[no]      = succ_train[comm].reshape(1,4,1)
    
for no, comm in enumerate(test_abundance):
    test_abundance_data[no]  = test_abundance[comm].reshape(1,4,25)
    test_butyrate[no]        = buty_test[comm].reshape(1,4,1)
    test_acetate[no]         = ace_test[comm].reshape(1,4,1)
    test_lactate[no]         = lac_test[comm].reshape(1,4,1)
    test_succinate[no]       = succ_test[comm].reshape(1,4,1)

In [6]:
normalize_all = False  #Normalize with respect to the entire dataset

seq_len = 3

#Concatenating for data normalization
X_abundance   = np.concatenate((train_abundance_data,test_abundance_data), axis=0)
X_buty        = np.concatenate((train_butyrate,test_butyrate), axis=0)
X_ace         = np.concatenate((train_acetate,test_acetate), axis=0)
X_lac         = np.concatenate((train_lactate,test_lactate), axis=0)
X_succ        = np.concatenate((train_succinate,test_succinate), axis=0)

if normalize_all:
    X_abun_metab  = np.concatenate((X_abundance, X_buty, X_ace, X_lac, X_succ), axis=2)
else:
    X_abun_metab  = np.concatenate((train_abundance_data, train_butyrate, train_acetate, train_lactate, train_succinate), axis=2)
    
Xm_abun_metab = np.mean(X_abun_metab,0)
Xv_abun_metab = 4*np.std(X_abun_metab,0)
Xv_abun_metab[Xv_abun_metab == 0] = 1.

training_data = np.concatenate((train_abundance_data, train_butyrate, train_acetate, train_lactate, train_succinate), axis=2)
testing_data  = np.concatenate((test_abundance_data, test_butyrate, test_acetate, test_lactate, test_succinate), axis=2)

In [7]:
Xm_train = np.tile(Xm_abun_metab,(train_abundance_data.shape[0],1,1))
Xv_train = np.tile(Xv_abun_metab,(train_abundance_data.shape[0],1,1))

Xm_test  = np.tile(Xm_abun_metab,(test_abundance_data.shape[0],1,1))
Xv_test  = np.tile(Xv_abun_metab,(test_abundance_data.shape[0],1,1))

train_data_red = (training_data - Xm_train)/Xv_train
test_data_red  = (testing_data - Xm_test)/Xv_test

X_train, Y_train = train_data_red[:,:-1,:], train_data_red[:,1:,:]
X_test, Y_test   = test_data_red[:,:-1,:], test_data_red[:,1:,:]

x_train, y_train = torch.from_numpy(X_train).float().to(device), torch.from_numpy(Y_train).float().to(device)
x_test, y_test   = torch.from_numpy(X_test).float().to(device), torch.from_numpy(Y_test).float().to(device)

### Define LSTM

In [8]:
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Model, self).__init__()
        self.input_size  = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.lstm        = nn.LSTMCell(self.input_size, self.hidden_size)
        self.linear      = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, inputs, future=0, y=None):
        outputs = []
        batch_size = inputs

        # reset the state of LSTM
        h_t = torch.zeros(inputs.size(0), self.hidden_size, dtype=torch.float32).to(device)
        c_t = torch.zeros(inputs.size(0), self.hidden_size, dtype=torch.float32).to(device)

        for i, input_t in enumerate(inputs.chunk(inputs.size(1), dim=1)):
            h_t, c_t   = self.lstm(torch.squeeze(input_t,1), (h_t, c_t))
            output     = self.linear(h_t)
            outputs   += [output]
            
        for i in range(future):
            if y is not None and random.random() > 0.5:
                output = torch.squeeze(y[:,[i],:], 1)  # teacher forcing
            h_t, c_t   = self.lstm(output, (h_t, c_t))
            output     = self.linear(h_t)
            outputs   += [output]
            
        outputs = torch.stack(outputs, 1).squeeze(2)
        return outputs

### Model hyperparameters

In [9]:
batch_size     = 10
lr             = 5e-3
lr_decay       = 0.25
decay_interval = 12
iteration      = 200

batch_size, decay_interval, iteration = map(int, [batch_size, decay_interval, iteration])
lr, lr_decay                          = map(float, [lr, lr_decay])

### Generate Batch Data

In [10]:
def generate_batch_data(x, y, batch_size):
    for batch, i in enumerate(range(0, len(x), batch_size)):
        x_batch     = x[i : i + batch_size]
        y_batch     = y[i : i + batch_size]
        yield x_batch, y_batch, batch

### Prediction function

In [11]:
def predict(model, x_batch, y_batch, seq_len, do_teacher_forcing):
    b_size = x_batch.size(0)
    f_size = x_batch.size(2)
    
    if do_teacher_forcing:
        future = random.randint(1, int(seq_len) // 2)
        limit  = x_batch.size(1) - future
        y_pred = model(x_batch[:, :limit].view(b_size,-1,f_size), future=future, y=y_batch[:, limit:].view(b_size,-1,f_size))
    else:
        future = 0
        y_pred = model(x_batch)
    
    return y_pred

### Create and Train Model

In [12]:
torch.manual_seed(1234)

model   = Model(29,4096,29).to(device)

stop
dir_output = ('output/')
os.makedirs(dir_output, exist_ok=True)

dir_output = ('output/Abundance/')
os.makedirs(dir_output, exist_ok=True)

dir_output = ('output/Metabolites/')
os.makedirs(dir_output, exist_ok=True)

dir_output = ('output/Community/Abundance/')
os.makedirs(dir_output, exist_ok=True)

dir_output = ('output/Community/Metabolites/')
os.makedirs(dir_output, exist_ok=True)

dir_output = ('output/heatmap/')
os.makedirs(dir_output, exist_ok=True)

optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
loss_fn   = nn.MSELoss()

print('Training...')
print('Epoch \t Time(sec) \t Loss_train \t Loss_test \t Learning_rate')

start = timeit.default_timer()

train_losses, test_losses = [], []

do_teacher_forcing = True
for epoch in range(iteration):
    if (epoch+1) % decay_interval == 0:
        if epoch <= 99:
            optimizer.param_groups[0]['lr'] *= lr_decay

    futures = []
    
    train_loss = 0
    for x_batch, y_batch, batch in generate_batch_data(x_train, y_train, batch_size):
        y_pred = predict(model, x_batch, y_batch, seq_len, do_teacher_forcing)
        optimizer.zero_grad()
        loss = loss_fn(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        
    train_losses.append(train_loss)
    
    model.eval()
    test_loss = 0
    actual, predicted = [], []
    for x_batch, y_batch, batch in generate_batch_data(x_test[:,0:1,:], y_test, batch_size):
        y_pred = model(x_batch, future=seq_len-1)
        y_pred = (y_pred[:, -len(y_batch) :] if y_pred.shape[1] > y_batch.shape[1] else y_pred)
        loss = loss_fn(y_pred, y_batch)
        test_loss += loss.item()
        actual += torch.squeeze(y_batch[:, -1]).data.cpu().numpy().tolist()
        predicted += torch.squeeze(y_pred[:, -1]).data.cpu().numpy().tolist()
        
    model.train()
    
    lr_rate = optimizer.param_groups[0]['lr']

    end  = timeit.default_timer()
    time = end - start

    print('%d \t %.4f \t %.4f \t %.4f \t %.6f' %(epoch, time, train_loss, test_loss, lr_rate))

NameError: name 'stop' is not defined

In [14]:
sum([p.numel() for p in model.parameters()])

67735581

In [ ]:
A = torch.zeros(3,3)
A[:, 0] = torch.tensor(np.array([3,3,3]), dtype=torch.float32)
A

### Plotting abundance data

In [ ]:
for ind_to_study in range(seq_len):
    for is_train in list([True, False]):
        
        Xm_ = Xm_abun_metab[ind_to_study+1]
        Xv_ = Xv_abun_metab[ind_to_study+1]

        model.eval()
        actual, predicted = [], []

        if is_train:
            for x_batch, y_batch, batch in generate_batch_data(x_train[:,0:1,:], y_train, batch_size):
                y_pred     = model(x_batch, future=seq_len-1)
                y_pred     = (y_pred[:, -len(y_batch) :] if y_pred.shape[1] > y_batch.shape[1] else y_pred)
                actual    += torch.squeeze(y_batch[:, ind_to_study]).data.cpu().numpy().reshape(y_pred.size(0),-1).tolist()
                predicted += torch.squeeze(y_pred[:, ind_to_study]).data.cpu().numpy().reshape(y_pred.size(0),-1).tolist()
                plt_str    = 'Training'
        else:
            for x_batch, y_batch, batch in generate_batch_data(x_test[:,0:1,:], y_test, batch_size):
                y_pred     = model(x_batch, future=seq_len-1)
                y_pred     = (y_pred[:, -len(y_batch) :] if y_pred.shape[1] > y_batch.shape[1] else y_pred)
                actual    += torch.squeeze(y_batch[:, ind_to_study]).data.cpu().numpy().reshape(y_pred.size(0),-1).tolist()
                predicted += torch.squeeze(y_pred[:, ind_to_study]).data.cpu().numpy().reshape(y_pred.size(0),-1).tolist()
                plt_str    = 'Testing'

        act_arr  = np.array(actual)
        pred_arr = np.array(predicted)
        
        Act  = (act_arr[:,:-4]*Xv_[:-4] + Xm_[:-4]).reshape(-1)
        Pred = (pred_arr[:,:-4]*Xv_[:-4] + Xm_[:-4]).reshape(-1)
        Pred[Pred<0] = 0.

        slope, intercept, r, p, se = stats.linregress(Act, Pred)
        
        print('Coefficient of determination: %.2f' %(r**2))

        x_vals = np.linspace(min(Act),max(Act),50)
        y_vals = slope*x_vals + intercept

        params = {'legend.fontsize': 24,
                  'figure.figsize' : (10, 10),
                 'axes.labelsize'  : 24,
                 'axes.titlesize'  : 24,
                 'xtick.labelsize' : 22,
                 'ytick.labelsize' : 22,
                 'pdf.fonttype'    : 42}
        plt.rcParams.update(params)


        plt.plot(Act, Pred, 'or')
        plt.plot(x_vals, y_vals, color='black', linewidth = 3)
        plt.plot(x_vals, x_vals, '--b', linewidth = 3)
        plt.axis('equal')
        plt.axis('square')
        plt.title(plt_str+' set: At t = '+str(int((1+ind_to_study)*16)) + ' hrs')
        plt.xlabel('True values')
        plt.ylabel('Predicted values')
        plt.legend(['Predicted abundance','Linear fit: slope = ' + format(slope,'.2f') + ', R2: ' + format(r**2,'.2f'),'unity slope'])
        plt.savefig('output/Abundance/'+plt_str+'_'+str(int((ind_to_study+1)*16))+'.pdf')
        plt.show()

### Plotting Metabolite Concentrations

In [ ]:
metab_list = ['Butyrate', 'Acetate', 'Lactate', 'Succinate']

for no, metab_name in enumerate(metab_list):
    metab_ind = no-4
    
    for ind_to_study in range(seq_len):
        for is_train in list([True, False]):

            Xm_ = Xm_abun_metab[ind_to_study+1]
            Xv_ = Xv_abun_metab[ind_to_study+1]

            model.eval()
            actual, predicted = [], []

            if is_train:
                for x_batch, y_batch, batch in generate_batch_data(x_train[:,0:1,:], y_train, batch_size):
                    y_pred     = model(x_batch, future=seq_len-1)
                    y_pred     = (y_pred[:, -len(y_batch) :] if y_pred.shape[1] > y_batch.shape[1] else y_pred)
                    actual    += torch.squeeze(y_batch[:, ind_to_study]).data.cpu().numpy().reshape(y_pred.size(0),-1).tolist()
                    predicted += torch.squeeze(y_pred[:, ind_to_study]).data.cpu().numpy().reshape(y_pred.size(0),-1).tolist()
                    plt_str    = 'Training_' + metab_name
            else:
                for x_batch, y_batch, batch in generate_batch_data(x_test[:,0:1,:], y_test, batch_size):
                    y_pred     = model(x_batch, future=seq_len-1)
                    y_pred     = (y_pred[:, -len(y_batch) :] if y_pred.shape[1] > y_batch.shape[1] else y_pred)
                    actual    += torch.squeeze(y_batch[:, ind_to_study]).data.cpu().numpy().tolist()
                    predicted += torch.squeeze(y_pred[:, ind_to_study]).data.cpu().numpy().tolist()
                    plt_str    = 'Testing_' + metab_name

            act_arr  = np.array(actual)
            pred_arr = np.array(predicted)

            Act  = (act_arr[:,metab_ind]*Xv_[metab_ind] + Xm_[metab_ind]).reshape(-1)
            Pred = (pred_arr[:,metab_ind]*Xv_[metab_ind] + Xm_[metab_ind]).reshape(-1)
            Pred[Pred<0] = 0.

            slope, intercept, r, p, se = stats.linregress(Act, Pred)
        
            print('Coefficient of determination: %.2f' %(r**2))

            x_vals = np.linspace(min(Act),max(Act),50)
            y_vals = slope*x_vals + intercept

            params = {'legend.fontsize': 24,
                      'figure.figsize' : (10, 10),
                     'axes.labelsize'  : 24,
                     'axes.titlesize'  : 24,
                     'xtick.labelsize' : 22,
                     'ytick.labelsize' : 22,
                     'pdf.fonttype'    : 42}
            plt.rcParams.update(params)


            plt.plot(Act, Pred, 'or')
            plt.plot(x_vals, y_vals, color='black', linewidth = 3)
            plt.plot(x_vals, x_vals, '--b', linewidth = 3)
            plt.axis('equal')
            plt.axis('square')
            plt.title(plt_str+' set: At t = '+str(int((1+ind_to_study)*16)) + ' hrs')
            plt.xlabel('True values')
            plt.ylabel('Predicted values')
            plt.legend(['Predicted concentration_' + metab_name,'Linear fit: slope = ' + format(slope,'.2f') + ', R2: ' + format(r**2,'.2f'),'unity slope'])
            plt.savefig('output/Metabolites/'+plt_str+'_'+str(int((ind_to_study+1)*16))+'.pdf')
            plt.show()

## Community-wise performance

In [ ]:
orderlist=['PC-BV-BT-BC-BP-EL-FP-CH-AC-BH-CG-ER-RI-DF', 
           'BV-BF-BO-BT-BU-DP-BL-BA-BP-EL-FP-CH-AC-BH-CG-ER-RI', 
           'PJ-BO-BT-BC-BY-BA-CA-EL-FP-CH-AC-BH-CG-CC-DF', 
           'PC-PJ-BF-BO-BU-DP-BA-EL-CH-BH-CG', 
           'PC-PJ-BV-BF-BT-BL-CA-EL-FP-CH-BH-CC-DF', 
           'PJ-BC-BY-BA-CA-EL-FP-AC-ER-CC-DF', 
           'PJ-BV-BO-BT-BY-BU-CA-EL-AC-CG-DF', 
           'PC-PJ-BT-BY-BU-DP-BA-CA-FP-CH-BH-CG-ER-RI-CC-DL-DF', 
           'BV-BU-DP-BA-CA-EL-FP-CH-CG-RI-DL-DF', 
           'PC-PJ-BF-BC-BU-DP-CA-EL-FP-CH-AC-CG', 
           'BO-BC-BY-BU-BA-BP-CH-AC-CG-CC-DL', 
           'PC-BV-BF-BT-BL-CA-CH-BH-CG-ER-RI-CC-DL', 
           'BC-BU-DP-BL-BP-CA-EL-FP-BH-DL-DF', 
           'PJ-BF-BC-BU-BP-CA-EL-FP-CH-AC-CG', 
           'BO-BC-BY-BU-BA-CA-EL-AC-RI-DL-DF', 
           'BV-BF-BY-BU-DP-BL-CA-EL-CH-BH-ER', 
           'BV-BF-BC-BY-BU-DP-BL-EL-CH-AC-CG', 
           'PJ-BF-DP-BL-BA-BP-CA-EL-BH-ER-DF', 
           'PJ-BV-BF-BT-BC-BY-BU-CH-AC-RI-DF', 
           'PC-PJ-BC-BY-BU-CA-EL-AC-BH-ER-RI', 
           'BY-BU-BP-CA-EL-FP-AC-BH-CG-ER-RI', 
           'PJ-BV-BF-BO-BU-DP-BL-CA-EL-FP-CH-AC', 
           'PC-BF-BO-BT-BC-BU-BP-CA-EL-AC-BH-RI', 
           'BF-BT-BC-BY-BL-BA-BP-EL-FP-AC-BH-CG-ER', 
           'PC-BF-BT-BC-BU-DP-BL-BA-FP-AC-CC-DL-DF', 
           'BC-BU-DP-BA-BP-CA-FP-CG-RI-CC-DF', 
           'PC-PJ-BV-BO-BC-BY-DP-BL-BA-BP-CA-EL-AC-BH-CG-ER-RI', 
           'PC-PJ-BV-BO-BY-BU-DP-BL-CA-EL-FP-CH-BH-CG-CC-DL', 
           'BV-BF-BL-CA-FP-CH-AC-BH-ER-RI-DF', 
           'PC-PJ-BO-BT-BC-BL-EL-FP-BH-CG-RI-DL', 
           'PJ-BV-BO-BC-BU-DP-BL-EL-CH-CG-RI', 
           'PC-PJ-BF-BT-BC-BY-DP-BL-BA-FP-AC-BH-RI-CC-DL-DF', 
           'PC-PJ-BV-BF-BT-BC-DP-CA-EL-CH-AC-BH-CG-RI-DF', 
           'BF-BU-DP-BL-BA-CA-EL-FP-AC-CG-RI', 
           'PC-BV-BF-BO-BC-DP-BL-EL-CH-BH-CG-RI', 
           'PC-BV-BF-BO-CA-EL-FP-CH-AC-BH-ER', 
           'PJ-BV-BY-BA-EL-FP-CH-BH-ER-DL-DF', 
           'PJ-BV-BF-BU-DP-BL-BA-EL-RI-DL-DF', 
           'BV-BO-BC-BY-BL-BA-FP-AC-BH-CG-DL-DF', 
           'PJ-BY-BU-EL-FP-CH-AC-BH-CG-CC-DF', 
           'PC-PJ-BO-BT-BL-BP-CH-AC-BH-CG-CC-DF', 
           'PJ-BO-BT-BU-BP-CA-EL-CH-AC-BH-RI', 
           'PC-PJ-BF-BU-DP-BA-CA-FP-CH-AC-RI-CC-DL', 
           'PJ-BV-BO-BT-BY-BU-DP-BL-CA-BH-CG-ER-CC-DL', 
           'BV-BF-BO-BY-BU-DP-CA-EL-CH-RI-DF', 
           'BF-BO-BT-BC-BU-CA-EL-AC-ER-RI-DF', 
           'PJ-BF-BC-BU-BA-BP-CA-EL-AC-BH-DL', 
           'PC-PJ-BV-BF-DP-EL-FP-CH-AC-ER-CC-DF', 
           'PC-PJ-BT-BC-BU-DP-BL-BP-CA-EL-RI-CC-DF', 
           'PC-BF-BT-BC-BU-DP-BL-CH-BH-RI-CC', 
           'PC-PJ-BV-BT-BC-BA-CA-CH-AC-ER-RI-CC-DL', 
           'BF-BO-BC-BL-CA-EL-BH-CG-RI-CC-DL', 
           'BY-BU-DP-BL-CA-EL-CH-BH-CG-CC-DF', 
           'PJ-BV-BF-BT-BC-BY-DP-CH-AC-BH-CC-DF', 
           'PC-PJ-BF-BT-BC-BU-DP-BL-CH-AC-BH-CG-RI-DF', 
           'PJ-BV-BO-BT-BC-BY-BP-CA-EL-FP-AC-BH-CG', 
           'PJ-BC-BP-EL-FP-AC-BH-ER-RI-CC-DF', 
           'BV-BO-BC-BU-DP-BP-EL-FP-AC-RI-DL-DF', 
           'PC-BF-BO-BU-BA-BP-CA-FP-CG-CC-DL-DF', 
           'BC-BU-BL-CA-EL-FP-AC-ER-RI-CC-DL', 
           'PC-PJ-BV-BF-BC-BY-BU-DP-BL-BA-BP-CA-FP-BH-ER-RI-CC-DL-DF', 
           'PC-PJ-BV-BO-BC-BY-BU-BP-EL-FP-AC-BH-CG-ER', 
           'PC-BV-BF-BY-BU-EL-FP-CH-AC-ER-DF', 
           'PC-BF-BO-BT-BC-BY-DP-BL-BP-CA-FP-CH-AC-BH-ER-RI-DL-DF', 
           'BF-BC-BY-BA-CA-EL-FP-CH-AC-BH-ER-CC-DL-DF', 
           'BV-BO-BT-BY-BL-BA-BP-EL-FP-AC-BH-DL-DF', 
           'PC-PJ-BV-BF-BY-BU-BA-EL-FP-CH-AC-CG-RI', 
           'PC-PJ-BV-BO-BT-BY-BA-BP-EL-FP-CH-CG-DL', 
           'PC-PJ-BL-BA-BP-CA-EL-FP-CH-AC-ER-RI', 
           'PC-PJ-BF-BY-DP-BA-CA-CG-RI-CC-DL', 
           'PC-BO-BC-BU-DP-BL-CA-AC-BH-CG-RI-CC-DF', 
           'PC-PJ-BF-BO-BY-BU-AC-BH-RI-DL-DF', 
           'PJ-BV-BF-BO-BT-BC-BU-BA-CA-BH-ER-CC-DF', 
           'PC-PJ-BV-BO-BC-BU-BL-BA-CA-FP-CH-BH-CG-CC', 
           'PC-PJ-BC-BU-BL-BP-CA-EL-FP-AC-CG-DL', 
           'PJ-BF-BO-BU-BA-CA-EL-CH-BH-CG-RI-CC-DL', 
           'PC-BF-BT-DP-BL-CA-CH-AC-CG-ER-DF', 
           'PC-PJ-BV-BF-BY-BA-BP-CA-EL-BH-CG-ER-DL', 
           'PC-PJ-BO-BT-BC-BY-FP-CH-AC-ER-RI-DL-DF', 
           'BF-BO-BC-BY-BU-DP-BA-BP-FP-CH-AC-BH-ER-RI-CC-DL-DF', 
           'PJ-BV-BO-BU-CA-CH-AC-BH-CG-RI-DL-DF', 
           'PC-BF-BO-BC-BY-DP-BL-BA-BP-CA-EL-AC-BH-CG-RI-CC-DL', 
           'PJ-BO-BT-BC-CA-EL-FP-AC-BH-ER-RI', 
           'PJ-BO-BT-BC-BY-BU-BL-BP-FP-AC-BH-CG-RI-DF', 
           'PC-PJ-BC-BY-DP-BL-CA-FP-CH-BH-ER-RI', 
           'BF-BU-BL-BA-EL-FP-CH-BH-CG-RI-DL', 
           'PC-PJ-BO-BC-BY-BU-DP-BL-CA-EL-CH-AC-BH-CG-DL-DF', 
           'PC-PJ-BV-BF-BO-BU-CA-CH-AC-BH-CG', 
           'PJ-BV-BT-BC-BY-BU-DP-BA-BP-CH-AC-CG-CC-DF', 
           'PC-BT-BC-BY-DP-BL-BA-CA-FP-CH-BH-CG-ER-RI', 
           'PC-PJ-BF-BO-BU-DP-BA-CA-EL-FP-CH-AC-BH-ER-CC-DL-DF', 
           'PC-BV-BF-BO-BT-BU-DP-CA-EL-FP-BH-CG-DF', 
           'PJ-BF-BO-BT-BC-BY-BU-DP-BL-BA-CH-AC-CG-RI-CC-DF', 
           'PJ-BV-BO-BU-CA-EL-FP-CH-BH-CG-ER',
           'PC-PJ-BV-BF-BO-BT-BC-BY-BU-DP-BL-BA-BP-CA-EL-FP-CH-AC-BH-CG-ER-RI-CC-DL-DF']

red_comm    = np.array([0, 1, 9, 10, 15, 16, 18, 21, 23, 24, 26, 31, 32, 38, 40, 42, 47, 50, 53, 54, 57, 63, 64, 65, 70, 72, 76, 78, 79, 80, 81, 83, 86, 88, 90, 92, 93, 94])
blue_comm   = np.array([2, 35, 66, 41, 22, 55, 87, 28, 61, 62])
green_comm  = np.array([3, 4, 7, 8, 11, 12, 17, 25, 27, 29, 30, 34, 36, 37, 43, 44, 48, 49, 51, 52, 58, 60, 67, 69, 73, 75, 77, 84, 85, 89, 91])
purple_comm = np.array([68, 5, 39, 20, 56, 59])
orange_comm = np.array([6, 45, 13, 82, 19])
grey_comm   = np.array([33, 71, 74, 14, 46])

red_comm_inds_train, red_comm_inds_val = [], []
for ind in red_comm:
    tmp = orderlist[ind]
    if tmp in comms_training:
        red_comm_inds_train.append(comms_training.index(tmp))
    else:
        red_comm_inds_val.append(comms_validation.index(tmp))


blue_comm_inds_train, blue_comm_inds_val = [], []
for ind in blue_comm:
    tmp = orderlist[ind]
    if tmp in comms_training:
        blue_comm_inds_train.append(comms_training.index(tmp))
    else:
        blue_comm_inds_val.append(comms_validation.index(tmp))


green_comm_inds_train, green_comm_inds_val = [], []
for ind in green_comm:
    tmp = orderlist[ind]
    if tmp in comms_training:
        green_comm_inds_train.append(comms_training.index(tmp))
    else:
        green_comm_inds_val.append(comms_validation.index(tmp))


purple_comm_inds_train, purple_comm_inds_val = [], []
for ind in purple_comm:
    tmp = orderlist[ind]
    if tmp in comms_training:
        purple_comm_inds_train.append(comms_training.index(tmp))
    else:
        purple_comm_inds_val.append(comms_validation.index(tmp))


orange_comm_inds_train, orange_comm_inds_val = [], []
for ind in orange_comm:
    tmp = orderlist[ind]
    if tmp in comms_training:
        orange_comm_inds_train.append(comms_training.index(tmp))
    else:
        orange_comm_inds_val.append(comms_validation.index(tmp))
        
        
grey_comm_inds_train, grey_comm_inds_val = [], []
for ind in grey_comm:
    tmp = orderlist[ind]
    if tmp in comms_training:
        grey_comm_inds_train.append(comms_training.index(tmp))
    else:
        grey_comm_inds_val.append(comms_validation.index(tmp))
        
        
red_comm_x    = torch.cat((x_train[red_comm_inds_train],x_test[red_comm_inds_val]),dim=0)
red_comm_y    = torch.cat((y_train[red_comm_inds_train],y_test[red_comm_inds_val]),dim=0)

blue_comm_x   = torch.cat((x_train[blue_comm_inds_train],x_test[blue_comm_inds_val]),dim=0)
blue_comm_y   = torch.cat((y_train[blue_comm_inds_train],y_test[blue_comm_inds_val]),dim=0)

green_comm_x  = torch.cat((x_train[green_comm_inds_train],x_test[green_comm_inds_val]),dim=0)
green_comm_y  = torch.cat((y_train[green_comm_inds_train],y_test[green_comm_inds_val]),dim=0)

purple_comm_x = torch.cat((x_train[purple_comm_inds_train],x_test[purple_comm_inds_val]),dim=0)
purple_comm_y = torch.cat((y_train[purple_comm_inds_train],y_test[purple_comm_inds_val]),dim=0)

orange_comm_x = torch.cat((x_train[orange_comm_inds_train],x_test[orange_comm_inds_val]),dim=0)
orange_comm_y = torch.cat((y_train[orange_comm_inds_train],y_test[orange_comm_inds_val]),dim=0)

grey_comm_x   = torch.cat((x_train[grey_comm_inds_train],x_test[grey_comm_inds_val]),dim=0)
grey_comm_y   = torch.cat((y_train[grey_comm_inds_train],y_test[grey_comm_inds_val]),dim=0)

### Abundance prediction

In [ ]:
for ind_to_study in range(seq_len):
    Xm_ = Xm_abun_metab[ind_to_study+1]
    Xv_ = Xv_abun_metab[ind_to_study+1]

    model.eval()
    actual, predicted = [], []
    
    for x_batch, y_batch, batch in generate_batch_data(red_comm_x[:,0:1,:], red_comm_y, batch_size):
        y_pred     = model(x_batch, future=seq_len-1)
        y_pred     = (y_pred[:, -len(y_batch) :] if y_pred.shape[1] > y_batch.shape[1] else y_pred)
        actual    += torch.squeeze(y_batch[:, ind_to_study]).data.cpu().numpy().reshape(y_pred.size(0),-1).tolist()
        predicted += torch.squeeze(y_pred[:, ind_to_study]).data.cpu().numpy().reshape(y_pred.size(0),-1).tolist()
        plt_str    = 'Red Community'

    act_arr  = np.array(actual)
    pred_arr = np.array(predicted)

    Act  = (act_arr[:,:-4]*Xv_[:-4] + Xm_[:-4]).reshape(-1)
    Pred = (pred_arr[:,:-4]*Xv_[:-4] + Xm_[:-4]).reshape(-1)
    Pred[Pred<0] = 0.

    slope, intercept, r, p, se = stats.linregress(Act, Pred)

    print('Coefficient of determination: %.2f' %(r**2))

    x_vals = np.linspace(min(Act),max(Act),50)
    y_vals = slope*x_vals + intercept

    params = {'legend.fontsize': 24,
              'figure.figsize' : (10, 10),
             'axes.labelsize'  : 24,
             'axes.titlesize'  : 24,
             'xtick.labelsize' : 22,
             'ytick.labelsize' : 22,
             'pdf.fonttype'    : 42}
    plt.rcParams.update(params)


    plt.plot(Act, Pred, 'or')
    plt.plot(x_vals, y_vals, color='black', linewidth = 3)
    plt.plot(x_vals, x_vals, '--b', linewidth = 3)
    plt.axis('equal')
    plt.axis('square')
    plt.title(plt_str+' set: At t = '+str(int((1+ind_to_study)*16)) + ' hrs')
    plt.xlabel('True values')
    plt.ylabel('Predicted values')
    plt.legend(['Predicted abundance','Linear fit: slope = ' + format(slope,'.2f') + ', R2: ' + format(r**2,'.2f'),'unity slope'])
    plt.savefig('output/Community/Abundance/'+plt_str+'_'+str(int((ind_to_study+1)*16))+'.pdf')
    plt.show()

In [ ]:
for ind_to_study in range(seq_len):
    Xm_ = Xm_abun_metab[ind_to_study+1]
    Xv_ = Xv_abun_metab[ind_to_study+1]

    model.eval()
    actual, predicted = [], []
    
    for x_batch, y_batch, batch in generate_batch_data(blue_comm_x[:,0:1,:], blue_comm_y, batch_size):
        y_pred     = model(x_batch, future=seq_len-1)
        y_pred     = (y_pred[:, -len(y_batch) :] if y_pred.shape[1] > y_batch.shape[1] else y_pred)
        actual    += torch.squeeze(y_batch[:, ind_to_study]).data.cpu().numpy().reshape(y_pred.size(0),-1).tolist()
        predicted += torch.squeeze(y_pred[:, ind_to_study]).data.cpu().numpy().reshape(y_pred.size(0),-1).tolist()
        plt_str    = 'Blue Community'

    act_arr  = np.array(actual)
    pred_arr = np.array(predicted)
    
    Act  = (act_arr[:,:-4]*Xv_[:-4] + Xm_[:-4]).reshape(-1)
    Pred = (pred_arr[:,:-4]*Xv_[:-4] + Xm_[:-4]).reshape(-1)
    Pred[Pred<0] = 0.
    
    slope, intercept, r, p, se = stats.linregress(Act, Pred)
        
    print('Coefficient of determination: %.2f' %(r**2))

    x_vals = np.linspace(min(Act),max(Act),50)
    y_vals = slope*x_vals + intercept

    params = {'legend.fontsize': 24,
              'figure.figsize' : (10, 10),
             'axes.labelsize'  : 24,
             'axes.titlesize'  : 24,
             'xtick.labelsize' : 22,
             'ytick.labelsize' : 22,
             'pdf.fonttype'    : 42}
    plt.rcParams.update(params)


    plt.plot(Act, Pred, 'or')
    plt.plot(x_vals, y_vals, color='black', linewidth = 3)
    plt.plot(x_vals, x_vals, '--b', linewidth = 3)
    plt.axis('equal')
    plt.axis('square')
    plt.title(plt_str+' set: At t = '+str(int((1+ind_to_study)*16)) + ' hrs')
    plt.xlabel('True values')
    plt.ylabel('Predicted values')
    plt.legend(['Predicted abundance','Linear fit: slope = ' + format(slope,'.2f') + ', R2: ' + format(r**2,'.2f'),'unity slope'])
    plt.savefig('output/Community/Abundance/'+plt_str+'_'+str(int((ind_to_study+1)*16))+'.pdf')
    plt.show()

In [ ]:
for ind_to_study in range(seq_len):
    Xm_ = Xm_abun_metab[ind_to_study+1]
    Xv_ = Xv_abun_metab[ind_to_study+1]

    model.eval()
    actual, predicted = [], []
    
    for x_batch, y_batch, batch in generate_batch_data(green_comm_x[:,0:1,:], green_comm_y, batch_size):
        y_pred     = model(x_batch, future=seq_len-1)
        y_pred     = (y_pred[:, -len(y_batch) :] if y_pred.shape[1] > y_batch.shape[1] else y_pred)
        actual    += torch.squeeze(y_batch[:, ind_to_study]).data.cpu().numpy().reshape(y_pred.size(0),-1).tolist()
        predicted += torch.squeeze(y_pred[:, ind_to_study]).data.cpu().numpy().reshape(y_pred.size(0),-1).tolist()
        plt_str    = 'Green Community'

    act_arr  = np.array(actual)
    pred_arr = np.array(predicted)

    Act  = (act_arr[:,:-4]*Xv_[:-4] + Xm_[:-4]).reshape(-1)
    Pred = (pred_arr[:,:-4]*Xv_[:-4] + Xm_[:-4]).reshape(-1)
    Pred[Pred<0] = 0.

    slope, intercept, r, p, se = stats.linregress(Act, Pred)
        
    print('Coefficient of determination: %.2f' %(r**2))

    x_vals = np.linspace(min(Act),max(Act),50)
    y_vals = slope*x_vals + intercept

    params = {'legend.fontsize': 24,
              'figure.figsize' : (10, 10),
             'axes.labelsize'  : 24,
             'axes.titlesize'  : 24,
             'xtick.labelsize' : 22,
             'ytick.labelsize' : 22,
             'pdf.fonttype'    : 42}
    plt.rcParams.update(params)


    plt.plot(Act, Pred, 'or')
    plt.plot(x_vals, y_vals, color='black', linewidth = 3)
    plt.plot(x_vals, x_vals, '--b', linewidth = 3)
    plt.axis('equal')
    plt.axis('square')
    plt.title(plt_str+' set: At t = '+str(int((1+ind_to_study)*16)) + ' hrs')
    plt.xlabel('True values')
    plt.ylabel('Predicted values')
    plt.legend(['Predicted abundance','Linear fit: slope = ' + format(slope,'.2f') + ', R2: ' + format(r**2,'.2f'),'unity slope'])
    plt.savefig('output/Community/Abundance/'+plt_str+'_'+str(int((ind_to_study+1)*16))+'.pdf')
    plt.show()

In [ ]:
for ind_to_study in range(seq_len):
    Xm_ = Xm_abun_metab[ind_to_study+1]
    Xv_ = Xv_abun_metab[ind_to_study+1]

    model.eval()
    actual, predicted = [], []
    
    for x_batch, y_batch, batch in generate_batch_data(purple_comm_x[:,0:1,:], purple_comm_y, batch_size):
        y_pred     = model(x_batch, future=seq_len-1)
        y_pred     = (y_pred[:, -len(y_batch) :] if y_pred.shape[1] > y_batch.shape[1] else y_pred)
        actual    += torch.squeeze(y_batch[:, ind_to_study]).data.cpu().numpy().reshape(y_pred.size(0),-1).tolist()
        predicted += torch.squeeze(y_pred[:, ind_to_study]).data.cpu().numpy().reshape(y_pred.size(0),-1).tolist()
        plt_str    = 'Purple Community'
    
    act_arr  = np.array(actual)
    pred_arr = np.array(predicted)
    
    Act  = (act_arr[:,:-4]*Xv_[:-4] + Xm_[:-4]).reshape(-1)
    Pred = (pred_arr[:,:-4]*Xv_[:-4] + Xm_[:-4]).reshape(-1)
    Pred[Pred<0] = 0.

    slope, intercept, r, p, se = stats.linregress(Act, Pred)
        
    print('Coefficient of determination: %.2f' %(r**2))

    x_vals = np.linspace(min(Act),max(Act),50)
    y_vals = slope*x_vals + intercept

    params = {'legend.fontsize': 24,
              'figure.figsize' : (10, 10),
             'axes.labelsize'  : 24,
             'axes.titlesize'  : 24,
             'xtick.labelsize' : 22,
             'ytick.labelsize' : 22,
             'pdf.fonttype'    : 42}
    plt.rcParams.update(params)


    plt.plot(Act, Pred, 'or')
    plt.plot(x_vals, y_vals, color='black', linewidth = 3)
    plt.plot(x_vals, x_vals, '--b', linewidth = 3)
    plt.axis('equal')
    plt.axis('square')
    plt.title(plt_str+' set: At t = '+str(int((1+ind_to_study)*16)) + ' hrs')
    plt.xlabel('True values')
    plt.ylabel('Predicted values')
    plt.legend(['Predicted abundance','Linear fit: slope = ' + format(slope,'.2f') + ', R2: ' + format(r**2,'.2f'),'unity slope'])
    plt.savefig('output/Community/Abundance/'+plt_str+'_'+str(int((ind_to_study+1)*16))+'.pdf')
    plt.show()

In [ ]:
for ind_to_study in range(seq_len):
    Xm_ = Xm_abun_metab[ind_to_study+1]
    Xv_ = Xv_abun_metab[ind_to_study+1]

    model.eval()
    actual, predicted = [], []
    
    for x_batch, y_batch, batch in generate_batch_data(orange_comm_x[:,0:1,:], orange_comm_y, batch_size):
        y_pred     = model(x_batch, future=seq_len-1)
        y_pred     = (y_pred[:, -len(y_batch) :] if y_pred.shape[1] > y_batch.shape[1] else y_pred)
        actual    += torch.squeeze(y_batch[:, ind_to_study]).data.cpu().numpy().reshape(y_pred.size(0),-1).tolist()
        predicted += torch.squeeze(y_pred[:, ind_to_study]).data.cpu().numpy().reshape(y_pred.size(0),-1).tolist()
        plt_str    = 'Orange Community'

    act_arr  = np.array(actual)
    pred_arr = np.array(predicted)

    Act  = (act_arr[:,:-4]*Xv_[:-4] + Xm_[:-4]).reshape(-1)
    Pred = (pred_arr[:,:-4]*Xv_[:-4] + Xm_[:-4]).reshape(-1)
    Pred[Pred<0] = 0.

    slope, intercept, r, p, se = stats.linregress(Act, Pred)
        
    print('Coefficient of determination: %.2f' %(r**2))

    x_vals = np.linspace(min(Act),max(Act),50)
    y_vals = slope*x_vals + intercept

    params = {'legend.fontsize': 24,
              'figure.figsize' : (10, 10),
             'axes.labelsize'  : 24,
             'axes.titlesize'  : 24,
             'xtick.labelsize' : 22,
             'ytick.labelsize' : 22,
             'pdf.fonttype'    : 42}
    plt.rcParams.update(params)


    plt.plot(Act, Pred, 'or')
    plt.plot(x_vals, y_vals, color='black', linewidth = 3)
    plt.plot(x_vals, x_vals, '--b', linewidth = 3)
    plt.axis('equal')
    plt.axis('square')
    plt.title(plt_str+' set: At t = '+str(int((1+ind_to_study)*16)) + ' hrs')
    plt.xlabel('True values')
    plt.ylabel('Predicted values')
    plt.legend(['Predicted abundance','Linear fit: slope = ' + format(slope,'.2f') + ', R2: ' + format(r**2,'.2f'),'unity slope'])
    plt.savefig('output/Community/Abundance/'+plt_str+'_'+str(int((ind_to_study+1)*16))+'.pdf')
    plt.show()

In [ ]:
for ind_to_study in range(seq_len):
    Xm_ = Xm_abun_metab[ind_to_study+1]
    Xv_ = Xv_abun_metab[ind_to_study+1]

    model.eval()
    actual, predicted = [], []
    
    for x_batch, y_batch, batch in generate_batch_data(grey_comm_x[:,0:1,:], grey_comm_y, batch_size):
        y_pred     = model(x_batch, future=seq_len-1)
        y_pred     = (y_pred[:, -len(y_batch) :] if y_pred.shape[1] > y_batch.shape[1] else y_pred)
        actual    += torch.squeeze(y_batch[:, ind_to_study]).data.cpu().numpy().reshape(y_pred.size(0),-1).tolist()
        predicted += torch.squeeze(y_pred[:, ind_to_study]).data.cpu().numpy().reshape(y_pred.size(0),-1).tolist()
        plt_str    = 'Grey Community'

    act_arr  = np.array(actual)
    pred_arr = np.array(predicted)

    Act  = (act_arr[:,:-4]*Xv_[:-4] + Xm_[:-4]).reshape(-1)
    Pred = (pred_arr[:,:-4]*Xv_[:-4] + Xm_[:-4]).reshape(-1)
    Pred[Pred<0] = 0.

    slope, intercept, r, p, se = stats.linregress(Act, Pred)
        
    print('Coefficient of determination: %.2f' %(r**2))

    x_vals = np.linspace(min(Act),max(Act),50)
    y_vals = slope*x_vals + intercept

    params = {'legend.fontsize': 24,
              'figure.figsize' : (10, 10),
             'axes.labelsize'  : 24,
             'axes.titlesize'  : 24,
             'xtick.labelsize' : 22,
             'ytick.labelsize' : 22,
             'pdf.fonttype'    : 42}
    plt.rcParams.update(params)


    plt.plot(Act, Pred, 'or')
    plt.plot(x_vals, y_vals, color='black', linewidth = 3)
    plt.plot(x_vals, x_vals, '--b', linewidth = 3)
    plt.axis('equal')
    plt.axis('square')
    plt.title(plt_str+' set: At t = '+str(int((1+ind_to_study)*16)) + ' hrs')
    plt.xlabel('True values')
    plt.ylabel('Predicted values')
    plt.legend(['Predicted abundance','Linear fit: slope = ' + format(slope,'.2f') + ', R2: ' + format(r**2,'.2f'),'unity slope'])
    plt.savefig('output/Community/Abundance/'+plt_str+'_'+str(int((ind_to_study+1)*16))+'.pdf')
    plt.show()

### Metabolite concentration prediction

In [ ]:
metab_list = ['Butyrate', 'Acetate', 'Lactate', 'Succinate']

for no, metab_name in enumerate(metab_list):
    metab_ind = no-4
    
    for ind_to_study in range(seq_len):
        Xm_ = Xm_abun_metab[ind_to_study+1]
        Xv_ = Xv_abun_metab[ind_to_study+1]

        model.eval()
        actual, predicted = [], []

        for x_batch, y_batch, batch in generate_batch_data(red_comm_x[:,0:1,:], red_comm_y, batch_size):
            y_pred     = model(x_batch, future=seq_len-1)
            y_pred     = (y_pred[:, -len(y_batch) :] if y_pred.shape[1] > y_batch.shape[1] else y_pred)
            actual    += torch.squeeze(y_batch[:, ind_to_study]).data.cpu().numpy().reshape(y_pred.size(0),-1).tolist()
            predicted += torch.squeeze(y_pred[:, ind_to_study]).data.cpu().numpy().reshape(y_pred.size(0),-1).tolist()
            plt_str    = 'Red Community - ' + metab_name

        act_arr  = np.array(actual)
        pred_arr = np.array(predicted)

        Act  = (act_arr[:,metab_ind]*Xv_[metab_ind] + Xm_[metab_ind]).reshape(-1)
        Pred = (pred_arr[:,metab_ind]*Xv_[metab_ind] + Xm_[metab_ind]).reshape(-1)
        Pred[Pred<0] = 0.

        slope, intercept, r, p, se = stats.linregress(Act, Pred)
        
        print('Coefficient of determination: %.2f' %(r**2))

        x_vals = np.linspace(min(Act),max(Act),50)
        y_vals = slope*x_vals + intercept

        params = {'legend.fontsize': 24,
                  'figure.figsize' : (10, 10),
                 'axes.labelsize'  : 24,
                 'axes.titlesize'  : 24,
                 'xtick.labelsize' : 22,
                 'ytick.labelsize' : 22,
                 'pdf.fonttype'    : 42}
        plt.rcParams.update(params)


        plt.plot(Act, Pred, 'or')
        plt.plot(x_vals, y_vals, color='black', linewidth = 3)
        plt.plot(x_vals, x_vals, '--b', linewidth = 3)
        plt.axis('equal')
        plt.axis('square')
        plt.title(plt_str+' set: At t = '+str(int((1+ind_to_study)*16)) + ' hrs')
        plt.xlabel('True values')
        plt.ylabel('Predicted values')
        plt.legend(['Predicted concentration_' + metab_name,'Linear fit: slope = ' + format(slope,'.2f') + ', R2: ' + format(r**2,'.2f'),'unity slope'])
        plt.savefig('output/Community/Metabolites/'+plt_str+'_'+str(int((ind_to_study+1)*16))+'.pdf')
        plt.show()

In [ ]:
metab_list = ['Butyrate', 'Acetate', 'Lactate', 'Succinate']

for no, metab_name in enumerate(metab_list):
    metab_ind = no-4
    
    for ind_to_study in range(seq_len):
        Xm_ = Xm_abun_metab[ind_to_study+1]
        Xv_ = Xv_abun_metab[ind_to_study+1]

        model.eval()
        actual, predicted = [], []

        for x_batch, y_batch, batch in generate_batch_data(blue_comm_x[:,0:1,:], blue_comm_y, batch_size):
            y_pred     = model(x_batch, future=seq_len-1)
            y_pred     = (y_pred[:, -len(y_batch) :] if y_pred.shape[1] > y_batch.shape[1] else y_pred)
            actual    += torch.squeeze(y_batch[:, ind_to_study]).data.cpu().numpy().reshape(y_pred.size(0),-1).tolist()
            predicted += torch.squeeze(y_pred[:, ind_to_study]).data.cpu().numpy().reshape(y_pred.size(0),-1).tolist()
            plt_str    = 'Blue Community - ' + metab_name

        act_arr  = np.array(actual)
        pred_arr = np.array(predicted)

        Act  = (act_arr[:,metab_ind]*Xv_[metab_ind] + Xm_[metab_ind]).reshape(-1)
        Pred = (pred_arr[:,metab_ind]*Xv_[metab_ind] + Xm_[metab_ind]).reshape(-1)
        Pred[Pred<0] = 0.

        slope, intercept, r, p, se = stats.linregress(Act, Pred)
        
        print('Coefficient of determination: %.2f' %(r**2))

        x_vals = np.linspace(min(Act),max(Act),50)
        y_vals = slope*x_vals + intercept

        params = {'legend.fontsize': 24,
                  'figure.figsize' : (10, 10),
                 'axes.labelsize'  : 24,
                 'axes.titlesize'  : 24,
                 'xtick.labelsize' : 22,
                 'ytick.labelsize' : 22,
                 'pdf.fonttype'    : 42}
        plt.rcParams.update(params)


        plt.plot(Act, Pred, 'or')
        plt.plot(x_vals, y_vals, color='black', linewidth = 3)
        plt.plot(x_vals, x_vals, '--b', linewidth = 3)
        plt.axis('equal')
        plt.axis('square')
        plt.title(plt_str+' set: At t = '+str(int((1+ind_to_study)*16)) + ' hrs')
        plt.xlabel('True values')
        plt.ylabel('Predicted values')
        plt.legend(['Predicted concentration_' + metab_name,'Linear fit: slope = ' + format(slope,'.2f') + ', R2: ' + format(r**2,'.2f'),'unity slope'])
        plt.savefig('output/Community/Metabolites/'+plt_str+'_'+str(int((ind_to_study+1)*16))+'.pdf')
        plt.show()

In [ ]:
metab_list = ['Butyrate', 'Acetate', 'Lactate', 'Succinate']

for no, metab_name in enumerate(metab_list):
    metab_ind = no-4
    
    for ind_to_study in range(seq_len):
        Xm_ = Xm_abun_metab[ind_to_study+1]
        Xv_ = Xv_abun_metab[ind_to_study+1]

        model.eval()
        actual, predicted = [], []

        for x_batch, y_batch, batch in generate_batch_data(green_comm_x[:,0:1,:], green_comm_y, batch_size):
            y_pred     = model(x_batch, future=seq_len-1)
            y_pred     = (y_pred[:, -len(y_batch) :] if y_pred.shape[1] > y_batch.shape[1] else y_pred)
            actual    += torch.squeeze(y_batch[:, ind_to_study]).data.cpu().numpy().reshape(y_pred.size(0),-1).tolist()
            predicted += torch.squeeze(y_pred[:, ind_to_study]).data.cpu().numpy().reshape(y_pred.size(0),-1).tolist()
            plt_str    = 'Green Community - ' + metab_name

        act_arr  = np.array(actual)
        pred_arr = np.array(predicted)

        Act  = (act_arr[:,metab_ind]*Xv_[metab_ind] + Xm_[metab_ind]).reshape(-1)
        Pred = (pred_arr[:,metab_ind]*Xv_[metab_ind] + Xm_[metab_ind]).reshape(-1)
        Pred[Pred<0] = 0.

        slope, intercept, r, p, se = stats.linregress(Act, Pred)
        
        print('Coefficient of determination: %.2f' %(r**2))

        x_vals = np.linspace(min(Act),max(Act),50)
        y_vals = slope*x_vals + intercept

        params = {'legend.fontsize': 24,
                  'figure.figsize' : (10, 10),
                 'axes.labelsize'  : 24,
                 'axes.titlesize'  : 24,
                 'xtick.labelsize' : 22,
                 'ytick.labelsize' : 22,
                 'pdf.fonttype'    : 42}
        plt.rcParams.update(params)


        plt.plot(Act, Pred, 'or')
        plt.plot(x_vals, y_vals, color='black', linewidth = 3)
        plt.plot(x_vals, x_vals, '--b', linewidth = 3)
        plt.axis('equal')
        plt.axis('square')
        plt.title(plt_str+' set: At t = '+str(int((1+ind_to_study)*16)) + ' hrs')
        plt.xlabel('True values')
        plt.ylabel('Predicted values')
        plt.legend(['Predicted concentration_' + metab_name,'Linear fit: slope = ' + format(slope,'.2f') + ', R2: ' + format(r**2,'.2f'),'unity slope'])
        plt.savefig('output/Community/Metabolites/'+plt_str+'_'+str(int((ind_to_study+1)*16))+'.pdf')
        plt.show()

In [ ]:
metab_list = ['Butyrate', 'Acetate', 'Lactate', 'Succinate']

for no, metab_name in enumerate(metab_list):
    metab_ind = no-4
    
    for ind_to_study in range(seq_len):
        Xm_ = Xm_abun_metab[ind_to_study+1]
        Xv_ = Xv_abun_metab[ind_to_study+1]

        model.eval()
        actual, predicted = [], []

        for x_batch, y_batch, batch in generate_batch_data(purple_comm_x[:,0:1,:], purple_comm_y, batch_size):
            y_pred     = model(x_batch, future=seq_len-1)
            y_pred     = (y_pred[:, -len(y_batch) :] if y_pred.shape[1] > y_batch.shape[1] else y_pred)
            actual    += torch.squeeze(y_batch[:, ind_to_study]).data.cpu().numpy().reshape(y_pred.size(0),-1).tolist()
            predicted += torch.squeeze(y_pred[:, ind_to_study]).data.cpu().numpy().reshape(y_pred.size(0),-1).tolist()
            plt_str    = 'Purple Community - ' + metab_name

        act_arr  = np.array(actual)
        pred_arr = np.array(predicted)

        Act  = (act_arr[:,metab_ind]*Xv_[metab_ind] + Xm_[metab_ind]).reshape(-1)
        Pred = (pred_arr[:,metab_ind]*Xv_[metab_ind] + Xm_[metab_ind]).reshape(-1)
        Pred[Pred<0] = 0.

        slope, intercept, r, p, se = stats.linregress(Act, Pred)
        
        print('Coefficient of determination: %.2f' %(r**2))

        x_vals = np.linspace(min(Act),max(Act),50)
        y_vals = slope*x_vals + intercept

        params = {'legend.fontsize': 24,
                  'figure.figsize' : (10, 10),
                 'axes.labelsize'  : 24,
                 'axes.titlesize'  : 24,
                 'xtick.labelsize' : 22,
                 'ytick.labelsize' : 22,
                 'pdf.fonttype'    : 42}
        plt.rcParams.update(params)


        plt.plot(Act, Pred, 'or')
        plt.plot(x_vals, y_vals, color='black', linewidth = 3)
        plt.plot(x_vals, x_vals, '--b', linewidth = 3)
        plt.axis('equal')
        plt.axis('square')
        plt.title(plt_str+' set: At t = '+str(int((1+ind_to_study)*16)) + ' hrs')
        plt.xlabel('True values')
        plt.ylabel('Predicted values')
        plt.legend(['Predicted concentration_' + metab_name,'Linear fit: slope = ' + format(slope,'.2f') + ', R2: ' + format(r**2,'.2f'),'unity slope'])
        plt.savefig('output/Community/Metabolites/'+plt_str+'_'+str(int((ind_to_study+1)*16))+'.pdf')
        plt.show()

In [ ]:
metab_list = ['Butyrate', 'Acetate', 'Lactate', 'Succinate']

for no, metab_name in enumerate(metab_list):
    metab_ind = no-4
    
    for ind_to_study in range(seq_len):
        Xm_ = Xm_abun_metab[ind_to_study+1]
        Xv_ = Xv_abun_metab[ind_to_study+1]

        model.eval()
        actual, predicted = [], []

        for x_batch, y_batch, batch in generate_batch_data(orange_comm_x[:,0:1,:], orange_comm_y, batch_size):
            y_pred     = model(x_batch, future=seq_len-1)
            y_pred     = (y_pred[:, -len(y_batch) :] if y_pred.shape[1] > y_batch.shape[1] else y_pred)
            actual    += torch.squeeze(y_batch[:, ind_to_study]).data.cpu().numpy().reshape(y_pred.size(0),-1).tolist()
            predicted += torch.squeeze(y_pred[:, ind_to_study]).data.cpu().numpy().reshape(y_pred.size(0),-1).tolist()
            plt_str    = 'Orange Community - ' + metab_name

        act_arr  = np.array(actual)
        pred_arr = np.array(predicted)

        Act  = (act_arr[:,metab_ind]*Xv_[metab_ind] + Xm_[metab_ind]).reshape(-1)
        Pred = (pred_arr[:,metab_ind]*Xv_[metab_ind] + Xm_[metab_ind]).reshape(-1)
        Pred[Pred<0] = 0.

        slope, intercept, r, p, se = stats.linregress(Act, Pred)
        
        print('Coefficient of determination: %.2f' %(r**2))

        x_vals = np.linspace(min(Act),max(Act),50)
        y_vals = slope*x_vals + intercept

        params = {'legend.fontsize': 24,
                  'figure.figsize' : (10, 10),
                 'axes.labelsize'  : 24,
                 'axes.titlesize'  : 24,
                 'xtick.labelsize' : 22,
                 'ytick.labelsize' : 22,
                 'pdf.fonttype'    : 42}
        plt.rcParams.update(params)


        plt.plot(Act, Pred, 'or')
        plt.plot(x_vals, y_vals, color='black', linewidth = 3)
        plt.plot(x_vals, x_vals, '--b', linewidth = 3)
        plt.axis('equal')
        plt.axis('square')
        plt.title(plt_str+' set: At t = '+str(int((1+ind_to_study)*16)) + ' hrs')
        plt.xlabel('True values')
        plt.ylabel('Predicted values')
        plt.legend(['Predicted concentration_' + metab_name,'Linear fit: slope = ' + format(slope,'.2f') + ', R2: ' + format(r**2,'.2f'),'unity slope'])
        plt.savefig('output/Community/Metabolites/'+plt_str+'_'+str(int((ind_to_study+1)*16))+'.pdf')
        plt.show()

In [ ]:
metab_list = ['Butyrate', 'Acetate', 'Lactate', 'Succinate']

for no, metab_name in enumerate(metab_list):
    metab_ind = no-4
    
    for ind_to_study in range(seq_len):
        Xm_ = Xm_abun_metab[ind_to_study+1]
        Xv_ = Xv_abun_metab[ind_to_study+1]

        model.eval()
        actual, predicted = [], []

        for x_batch, y_batch, batch in generate_batch_data(grey_comm_x[:,0:1,:], grey_comm_y, batch_size):
            y_pred     = model(x_batch, future=seq_len-1)
            y_pred     = (y_pred[:, -len(y_batch) :] if y_pred.shape[1] > y_batch.shape[1] else y_pred)
            actual    += torch.squeeze(y_batch[:, ind_to_study]).data.cpu().numpy().reshape(y_pred.size(0),-1).tolist()
            predicted += torch.squeeze(y_pred[:, ind_to_study]).data.cpu().numpy().reshape(y_pred.size(0),-1).tolist()
            plt_str    = 'Grey Community - ' + metab_name

        act_arr  = np.array(actual)
        pred_arr = np.array(predicted)

        Act  = (act_arr[:,metab_ind]*Xv_[metab_ind] + Xm_[metab_ind]).reshape(-1)
        Pred = (pred_arr[:,metab_ind]*Xv_[metab_ind] + Xm_[metab_ind]).reshape(-1)
        Pred[Pred<0] = 0.

        slope, intercept, r, p, se = stats.linregress(Act, Pred)
        
        print('Coefficient of determination: %.2f' %(r**2))

        x_vals = np.linspace(min(Act),max(Act),50)
        y_vals = slope*x_vals + intercept

        params = {'legend.fontsize': 24,
                  'figure.figsize' : (10, 10),
                 'axes.labelsize'  : 24,
                 'axes.titlesize'  : 24,
                 'xtick.labelsize' : 22,
                 'ytick.labelsize' : 22,
                 'pdf.fonttype'    : 42}
        plt.rcParams.update(params)


        plt.plot(Act, Pred, 'or')
        plt.plot(x_vals, y_vals, color='black', linewidth = 3)
        plt.plot(x_vals, x_vals, '--b', linewidth = 3)
        plt.axis('equal')
        plt.axis('square')
        plt.title(plt_str+' set: At t = '+str(int((1+ind_to_study)*16)) + ' hrs')
        plt.xlabel('True values')
        plt.ylabel('Predicted values')
        plt.legend(['Predicted concentration_' + metab_name,'Linear fit: slope = ' + format(slope,'.2f') + ', R2: ' + format(r**2,'.2f'),'unity slope'])
        plt.savefig('output/Community/Metabolites/'+plt_str+'_'+str(int((ind_to_study+1)*16))+'.pdf')
        plt.show()

## Visualizing heat maps

### Butyrate

In [ ]:
buty_grad_list = {}

Xm__      = torch.tensor(Xm_abun_metab[0,:].reshape(1,1,-1)).to(device)
Xv__      = torch.tensor(Xv_abun_metab[0,:].reshape(1,1,-1)).to(device)

met_ind   = 25

for ind_to_study in range(seq_len):

    buty_grad_list[ind_to_study] = []
    
    for b in range(x_test.size(0)):
        model.eval()
        inp        = Variable(x_test[b:b+1,0:1,:],requires_grad=True)
        y_pred     = model(inp, future=seq_len-1)
        
        x_ini      = torch.squeeze(torch.squeeze(x_test[b:b+1,0:1,:]*Xv__ + Xm__,1),0).to('cpu').data.numpy()
        mask       = np.ones((25,))
        mask[x_ini[:25]<1e-6] = 0.
        
        y_pred[0,ind_to_study,met_ind].backward()
        grad_data = inp.grad.data
        
        buty_grad_list[ind_to_study].append((grad_data.to('cpu').numpy()[0,0,:25]*mask).tolist())

In [ ]:
hm = []

hm.append((Xv_abun_metab[0][:25]/Xv_abun_metab[0][met_ind])*buty_grad_list[0][-1])
hm.append((Xv_abun_metab[1][:25]/Xv_abun_metab[1][met_ind])*buty_grad_list[1][-1])
hm.append((Xv_abun_metab[2][:25]/Xv_abun_metab[2][met_ind])*buty_grad_list[2][-1])

params = {'legend.fontsize': 24,
          'figure.figsize' : (10, 10),
         'axes.labelsize'  : 24,
         'axes.titlesize'  : 24,
         'xtick.labelsize' : 22,
         'ytick.labelsize' : 22,
         'pdf.fonttype'    : 42}
plt.rcParams.update(params)

f, ax = plt.subplots(figsize=(25, 5))
x_axis_labels = ['PC','PJ','BV','BF','BO','BT','BC','BY','BU','DP','BL','BA','BP','CA','EL','FP','CH','AC','BH','CG','ER','RI','CC','DL','DF']
y_axis_labels = [16, 32, 48]

abs_v = np.max(np.array(hm))
ax.set_title('Butyrate')
ax = sns.heatmap(np.array(hm), xticklabels=x_axis_labels, yticklabels=y_axis_labels, cmap="bwr", vmin=-abs_v, vmax=abs_v)
plt.savefig('output/heatmap/butyrate.pdf')

### Acetate

In [ ]:
ace_grad_list = {}

Xm__      = torch.tensor(Xm_abun_metab[0,:].reshape(1,1,-1)).to(device)
Xv__      = torch.tensor(Xv_abun_metab[0,:].reshape(1,1,-1)).to(device)

met_ind   = 26

for ind_to_study in range(seq_len):

    ace_grad_list[ind_to_study] = []
    
    for b in range(x_test.size(0)):
        model.eval()
        inp        = Variable(x_test[b:b+1,0:1,:],requires_grad=True)
        y_pred     = model(inp, future=seq_len-1)
        
        x_ini      = torch.squeeze(torch.squeeze(x_test[b:b+1,0:1,:]*Xv__ + Xm__,1),0).to('cpu').data.numpy()
        mask       = np.ones((25,))
        mask[x_ini[:25]<1e-6] = 0.
        
        y_pred[0,ind_to_study,met_ind].backward()
        grad_data = inp.grad.data
        
        ace_grad_list[ind_to_study].append((grad_data.to('cpu').numpy()[0,0,:25]*mask).tolist())

In [ ]:
hm = []

hm.append((Xv_abun_metab[0][:25]/Xv_abun_metab[0][met_ind])*ace_grad_list[0][-1])
hm.append((Xv_abun_metab[1][:25]/Xv_abun_metab[1][met_ind])*ace_grad_list[1][-1])
hm.append((Xv_abun_metab[2][:25]/Xv_abun_metab[2][met_ind])*ace_grad_list[2][-1])

params = {'legend.fontsize': 24,
          'figure.figsize' : (10, 10),
         'axes.labelsize'  : 24,
         'axes.titlesize'  : 24,
         'xtick.labelsize' : 22,
         'ytick.labelsize' : 22,
         'pdf.fonttype'    : 42}
plt.rcParams.update(params)

f, ax = plt.subplots(figsize=(25, 5))
x_axis_labels = ['PC','PJ','BV','BF','BO','BT','BC','BY','BU','DP','BL','BA','BP','CA','EL','FP','CH','AC','BH','CG','ER','RI','CC','DL','DF']
y_axis_labels = [16, 32, 48]

abs_v = np.max(np.array(hm))
ax.set_title('Acetate')
ax = sns.heatmap(np.array(hm), xticklabels=x_axis_labels, yticklabels=y_axis_labels, cmap="bwr", vmin=-abs_v, vmax=abs_v)
plt.savefig('output/heatmap/acetate.pdf')

### Lactate

In [ ]:
lac_grad_list = {}

Xm__      = torch.tensor(Xm_abun_metab[0,:].reshape(1,1,-1)).to(device)
Xv__      = torch.tensor(Xv_abun_metab[0,:].reshape(1,1,-1)).to(device)

met_ind   = 27

for ind_to_study in range(seq_len):

    lac_grad_list[ind_to_study] = []
    
    for b in range(x_test.size(0)):
        model.eval()
        inp        = Variable(x_test[b:b+1,0:1,:],requires_grad=True)
        y_pred     = model(inp, future=seq_len-1)
        
        x_ini      = torch.squeeze(torch.squeeze(x_test[b:b+1,0:1,:]*Xv__ + Xm__,1),0).to('cpu').data.numpy()
        mask       = np.ones((25,))
        mask[x_ini[:25]<1e-6] = 0.
        
        y_pred[0,ind_to_study,met_ind].backward()
        grad_data = inp.grad.data
        
        lac_grad_list[ind_to_study].append((grad_data.to('cpu').numpy()[0,0,:25]*mask).tolist())

In [ ]:
hm = []

hm.append((Xv_abun_metab[0][:25]/Xv_abun_metab[0][met_ind])*lac_grad_list[0][-1])
hm.append((Xv_abun_metab[1][:25]/Xv_abun_metab[1][met_ind])*lac_grad_list[1][-1])
hm.append((Xv_abun_metab[2][:25]/Xv_abun_metab[2][met_ind])*lac_grad_list[2][-1])

params = {'legend.fontsize': 24,
          'figure.figsize' : (10, 10),
         'axes.labelsize'  : 24,
         'axes.titlesize'  : 24,
         'xtick.labelsize' : 22,
         'ytick.labelsize' : 22,
         'pdf.fonttype'    : 42}
plt.rcParams.update(params)

f, ax = plt.subplots(figsize=(25, 5))
x_axis_labels = ['PC','PJ','BV','BF','BO','BT','BC','BY','BU','DP','BL','BA','BP','CA','EL','FP','CH','AC','BH','CG','ER','RI','CC','DL','DF']
y_axis_labels = [16, 32, 48]

abs_v = np.max(np.array(hm))
ax.set_title('Lactate')
ax = sns.heatmap(np.array(hm), xticklabels=x_axis_labels, yticklabels=y_axis_labels, cmap="bwr", vmin=-abs_v, vmax=abs_v)
plt.savefig('output/heatmap/lactate.pdf')

### Succinate

In [ ]:
succ_grad_list = {}

Xm__      = torch.tensor(Xm_abun_metab[0,:].reshape(1,1,-1)).to(device)
Xv__      = torch.tensor(Xv_abun_metab[0,:].reshape(1,1,-1)).to(device)

met_ind   = 28

for ind_to_study in range(seq_len):

    succ_grad_list[ind_to_study] = []
    
    for b in range(x_test.size(0)):
        model.eval()
        inp        = Variable(x_test[b:b+1,0:1,:],requires_grad=True)
        y_pred     = model(inp, future=seq_len-1)
        
        x_ini      = torch.squeeze(torch.squeeze(x_test[b:b+1,0:1,:]*Xv__ + Xm__,1),0).to('cpu').data.numpy()
        mask       = np.ones((25,))
        mask[x_ini[:25]<1e-6] = 0.
        
        y_pred[0,ind_to_study,met_ind].backward()
        grad_data = inp.grad.data
        
        succ_grad_list[ind_to_study].append((grad_data.to('cpu').numpy()[0,0,:25]*mask).tolist())

In [ ]:
hm = []

hm.append((Xv_abun_metab[0][:25]/Xv_abun_metab[0][met_ind])*succ_grad_list[0][-1])
hm.append((Xv_abun_metab[1][:25]/Xv_abun_metab[1][met_ind])*succ_grad_list[1][-1])
hm.append((Xv_abun_metab[2][:25]/Xv_abun_metab[2][met_ind])*succ_grad_list[2][-1])

params = {'legend.fontsize': 24,
          'figure.figsize' : (10, 10),
         'axes.labelsize'  : 24,
         'axes.titlesize'  : 24,
         'xtick.labelsize' : 22,
         'ytick.labelsize' : 22,
         'pdf.fonttype'    : 42}
plt.rcParams.update(params)

f, ax = plt.subplots(figsize=(25, 5))
x_axis_labels = ['PC','PJ','BV','BF','BO','BT','BC','BY','BU','DP','BL','BA','BP','CA','EL','FP','CH','AC','BH','CG','ER','RI','CC','DL','DF']
y_axis_labels = [16, 32, 48]

abs_v = np.max(np.array(hm))
ax.set_title('Succinate')
ax = sns.heatmap(np.array(hm), xticklabels=x_axis_labels, yticklabels=y_axis_labels, cmap="bwr", vmin=-abs_v, vmax=abs_v)
plt.savefig('output/heatmap/succinate.pdf')

### Species-wise prediction performance

In [ ]:
sp_list = ['PC', 'PJ', 'BV', 'BF', 'BO', 'BT', \
           'BC', 'BY', 'BU', 'DP', 'BL', 'BA', \
           'BP', 'CA', 'EL', 'FP', 'CH', 'AC', \
           'BH', 'CG', 'ER', 'RI', 'CC', 'DL', 'DF']

for sp_ind, sp in enumerate(sp_list):
    for ind_to_study in range(seq_len):
        for is_train in list([True, False]):

            Xm_ = Xm_abun_metab[ind_to_study+1]
            Xv_ = Xv_abun_metab[ind_to_study+1]

            model.eval()
            actual, predicted = [], []

            if is_train:
                for x_batch, y_batch, batch in generate_batch_data(x_train[:,0:1,:], y_train, batch_size):
                    inds_inc   = np.where(abs((x_batch[:,0,sp_ind]*Xv_abun_metab[0,sp_ind] + Xm_abun_metab[0,sp_ind]).to('cpu').data.numpy())>1e-5)[0]
                    y_pred     = model(x_batch, future=seq_len-1)
                    y_pred     = (y_pred[:, -len(y_batch) :] if y_pred.shape[1] > y_batch.shape[1] else y_pred)
                    
                    if len(inds_inc) > 0:
                        actual    += y_batch[inds_inc, ind_to_study, sp_ind].data.cpu().numpy().tolist()
                        predicted += y_pred[inds_inc, ind_to_study, sp_ind].data.cpu().numpy().tolist()
                        plt_str    = 'Training'
            else:
                for x_batch, y_batch, batch in generate_batch_data(x_test[:,0:1,:], y_test, batch_size):
                    inds_inc   = np.where(abs((x_batch[:,0,sp_ind]*Xv_abun_metab[0,sp_ind] + Xm_abun_metab[0,sp_ind]).to('cpu').data.numpy())>1e-5)[0]
                    y_pred     = model(x_batch, future=seq_len-1)
                    y_pred     = (y_pred[:, -len(y_batch) :] if y_pred.shape[1] > y_batch.shape[1] else y_pred)
                    
                    if len(inds_inc) > 0:
                        actual    += y_batch[inds_inc, ind_to_study, sp_ind].data.cpu().numpy().tolist()
                        predicted += y_pred[inds_inc, ind_to_study, sp_ind].data.cpu().numpy().tolist()
                        plt_str    = 'Testing'

            act_arr  = np.array(actual)
            pred_arr = np.array(predicted)

            Act  = (act_arr*Xv_[sp_ind] + Xm_[sp_ind]).reshape(-1)
            Pred = (pred_arr*Xv_[sp_ind] + Xm_[sp_ind]).reshape(-1)
            Pred[Pred<0] = 0.

            slope, intercept, r, p, se = stats.linregress(Act, Pred)

            print('Coefficient of determination: %.2f' %(r**2))

            x_vals = np.linspace(min(Act),max(Act),50)
            y_vals = slope*x_vals + intercept

            params = {'legend.fontsize': 24,
                      'figure.figsize' : (10, 10),
                     'axes.labelsize'  : 24,
                     'axes.titlesize'  : 24,
                     'xtick.labelsize' : 22,
                     'ytick.labelsize' : 22,
                     'pdf.fonttype'    : 42}
            plt.rcParams.update(params)


            plt.plot(Act, Pred, 'or')
            plt.plot(x_vals, y_vals, color='black', linewidth = 3)
            plt.plot(x_vals, x_vals, '--b', linewidth = 3)
            plt.axis('equal')
            plt.axis('square')
            plt.title(plt_str+' set ('+sp+'): At t = '+str(int((1+ind_to_study)*16)) + ' hrs')
            plt.xlabel('True values')
            plt.ylabel('Predicted values')
            plt.legend(['Predicted abundance','Linear fit: slope = ' + format(slope,'.2f') + ', R2: ' + format(r**2,'.2f'),'unity slope'])
            plt.savefig('output/Abundance/'+plt_str+'_'+sp+'_'+str(int((ind_to_study+1)*16))+'.pdf')
            plt.show()

### Community-wise heatmaps

### Grey community heatmaps (Sample code)

In [ ]:
buty_grad_list = {}
ace_grad_list  = {}
lac_grad_list  = {}
succ_grad_list = {}

Xm__      = torch.tensor(Xm_abun_metab[0,:].reshape(1,1,-1)).to(device)
Xv__      = torch.tensor(Xv_abun_metab[0,:].reshape(1,1,-1)).to(device)

met_ind   = 25
for ind_to_study in range(seq_len):
    buty_grad_list[ind_to_study] = []
    for b in range(grey_comm_x.size(0)):
        model.eval()
        inp        = Variable(grey_comm_x[b:b+1,0:1,:],requires_grad=True)
        y_pred     = model(inp, future=seq_len-1)
        
        x_ini      = torch.squeeze(torch.squeeze(grey_comm_x[b:b+1,0:1,:]*Xv__ + Xm__,1),0).to('cpu').data.numpy()
        mask       = np.ones((25,))
        mask[x_ini[:25]<1e-6] = 0.
        
        y_pred[0,ind_to_study,met_ind].backward()
        grad_data = inp.grad.data
        
        buty_grad_list[ind_to_study].append((grad_data.to('cpu').numpy()[0,0,:25]*mask).tolist())
        
        
met_ind   = 26
for ind_to_study in range(seq_len):
    ace_grad_list[ind_to_study] = []
    for b in range(grey_comm_x.size(0)):
        model.eval()
        inp        = Variable(grey_comm_x[b:b+1,0:1,:],requires_grad=True)
        y_pred     = model(inp, future=seq_len-1)
        
        x_ini      = torch.squeeze(torch.squeeze(grey_comm_x[b:b+1,0:1,:]*Xv__ + Xm__,1),0).to('cpu').data.numpy()
        mask       = np.ones((25,))
        mask[x_ini[:25]<1e-6] = 0.
        
        y_pred[0,ind_to_study,met_ind].backward()
        grad_data = inp.grad.data
        
        ace_grad_list[ind_to_study].append((grad_data.to('cpu').numpy()[0,0,:25]*mask).tolist())
        
        
met_ind   = 27
for ind_to_study in range(seq_len):
    lac_grad_list[ind_to_study] = []
    for b in range(grey_comm_x.size(0)):
        model.eval()
        inp        = Variable(grey_comm_x[b:b+1,0:1,:],requires_grad=True)
        y_pred     = model(inp, future=seq_len-1)
        
        x_ini      = torch.squeeze(torch.squeeze(grey_comm_x[b:b+1,0:1,:]*Xv__ + Xm__,1),0).to('cpu').data.numpy()
        mask       = np.ones((25,))
        mask[x_ini[:25]<1e-6] = 0.
        
        y_pred[0,ind_to_study,met_ind].backward()
        grad_data = inp.grad.data
        
        lac_grad_list[ind_to_study].append((grad_data.to('cpu').numpy()[0,0,:25]*mask).tolist())
        
        
met_ind   = 28
for ind_to_study in range(seq_len):
    succ_grad_list[ind_to_study] = []
    for b in range(grey_comm_x.size(0)):
        model.eval()
        inp        = Variable(grey_comm_x[b:b+1,0:1,:],requires_grad=True)
        y_pred     = model(inp, future=seq_len-1)
        
        x_ini      = torch.squeeze(torch.squeeze(grey_comm_x[b:b+1,0:1,:]*Xv__ + Xm__,1),0).to('cpu').data.numpy()
        mask       = np.ones((25,))
        mask[x_ini[:25]<1e-6] = 0.
        
        y_pred[0,ind_to_study,met_ind].backward()
        grad_data = inp.grad.data
        
        succ_grad_list[ind_to_study].append((grad_data.to('cpu').numpy()[0,0,:25]*mask).tolist())

In [ ]:
dir_output = ('output/heatmap/GreyCommunity/')
os.makedirs(dir_output, exist_ok=True)

for b in range(grey_comm_x.size(0)):
    for met_ind in range(25,29):
        hm = []
        if met_ind == 25:
            hm.append((Xv_abun_metab[0][:25]/Xv_abun_metab[0][met_ind])*buty_grad_list[0][b])
            hm.append((Xv_abun_metab[1][:25]/Xv_abun_metab[1][met_ind])*buty_grad_list[1][b])
            hm.append((Xv_abun_metab[2][:25]/Xv_abun_metab[2][met_ind])*buty_grad_list[2][b])
            plt_str = 'Butyrate'
        elif met_ind == 26:
            hm.append((Xv_abun_metab[0][:25]/Xv_abun_metab[0][met_ind])*ace_grad_list[0][b])
            hm.append((Xv_abun_metab[1][:25]/Xv_abun_metab[1][met_ind])*ace_grad_list[1][b])
            hm.append((Xv_abun_metab[2][:25]/Xv_abun_metab[2][met_ind])*ace_grad_list[2][b])
            plt_str = 'Acetate'
        elif met_ind == 27:
            hm.append((Xv_abun_metab[0][:25]/Xv_abun_metab[0][met_ind])*lac_grad_list[0][b])
            hm.append((Xv_abun_metab[1][:25]/Xv_abun_metab[1][met_ind])*lac_grad_list[1][b])
            hm.append((Xv_abun_metab[2][:25]/Xv_abun_metab[2][met_ind])*lac_grad_list[2][b])
            plt_str = 'Lactate'
        else:
            hm.append((Xv_abun_metab[0][:25]/Xv_abun_metab[0][met_ind])*succ_grad_list[0][b])
            hm.append((Xv_abun_metab[1][:25]/Xv_abun_metab[1][met_ind])*succ_grad_list[1][b])
            hm.append((Xv_abun_metab[2][:25]/Xv_abun_metab[2][met_ind])*succ_grad_list[2][b])
            plt_str = 'Succinate'
            
            
        params = {'legend.fontsize': 24,
                  'figure.figsize' : (10, 10),
                 'axes.labelsize'  : 24,
                 'axes.titlesize'  : 24,
                 'xtick.labelsize' : 22,
                 'ytick.labelsize' : 22,
                 'pdf.fonttype'    : 42}
        plt.rcParams.update(params)
        
        f, ax = plt.subplots(figsize=(25, 5))
        x_axis_labels = ['PC','PJ','BV','BF','BO','BT','BC','BY','BU','DP','BL','BA','BP','CA','EL','FP','CH','AC','BH','CG','ER','RI','CC','DL','DF']
        y_axis_labels = [16, 32, 48]
        
        abs_v = np.max(np.array(hm))
        ax.set_title(plt_str + ': Example '+str(b+1))
        ax = sns.heatmap(np.array(hm), xticklabels=x_axis_labels, yticklabels=y_axis_labels, cmap="bwr", vmin=-abs_v, vmax=abs_v)
        plt.savefig('output/heatmap/GreyCommunity/example_'+str(b+1)+'_'+plt_str+'.pdf')